In [1]:
! pip install azureml-sdk

**Importing the dependencies**

In [2]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [6]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [7]:
print(resource_group)
print(workspace_name)
print(region)

mL-model-deployment-rag
deploy-sklearn-model-ws
Canada Central 


**Create a Resource Group from Azure Portal**

In [8]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-4c86fe0c7b
Deploying KeyVault with name deployskkeyvault2af71ba5.
Deploying StorageAccount with name deployskstorageff77fcd4d.
Deploying AppInsights with name deployskinsights4739270e.
Deployed AppInsights with name deployskinsights4739270e. Took 40.23 seconds.
Deploying Workspace with name deploy-sklearn-model-ws.
Deployed Workspace with name deploy-sklearn-model-ws. Took 22.44 seconds.
Workspace deploy-sklearn-model-ws created


In [9]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'

In [10]:
model_name='diabetes_prediction_model'

In [11]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


In [12]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [13]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [14]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [15]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\Sunsoft\AppData\Local\Temp\ipykernel_9260\3861565384.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-09-26 12:48:37+03:00 Creating Container Registry if not exists..
2024-09-26 12:58:38+03:00 Registering the environment..
2024-09-26 12:58:40+03:00 Building image..
2024-09-26 13:06:55+03:00 Generating deployment configuration..
2024-09-26 13:06:57+03:00 Submitting deployment to compute..
2024-09-26 13:07:03+03:00 Checking the status of deployment diabetes-prediction-service..
2024-09-26 13:08:19+03:00 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [16]:
scoring_uri = service.scoring_uri

In [17]:
scoring_uri

'http://bf14c7a9-c64a-416b-b7ca-8827b6b899b3.canadacentral.azurecontainer.io/score'

**Cleaning up all the created resources**

In [25]:
# Delete the service

service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-09-25 17:00:31+03:00 Check and wait for operation (a480cf2d-44a6-405e-96ad-aa725d4cbe1e) to finish.
2024-09-25 17:00:35+03:00 Deleting service..
2024-09-25 17:00:38+03:00 Deleting service entity.
Succeeded


In [26]:
# Specify the name of the registered model to delete
model_name = 'diabetes_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()


In [27]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)

1. Azure Account with appropriate subscription
2. Configuration file that has subscription_id and other details like resource_group name, etc
3. Create a resource group from Azure Portal
4. Create a workspace
5. Register the model in container registry
6. Deploy the model as endpoint
7. Once done, delete the created resources